In [10]:
## Subqueries in where clause


SELECT Student Sid,Sname
FROM Studen,Apply
WHERE Student.sId=Apply.sID AND major=='CS'

(Gonno get duplicates here )


OR

SELECT Sid,Sname
FROM Student
WHERE Sid IN (SELECT Sid FROM Apply WHERE major=='CS')



# who applied to CS but not EE

SELECT sID,sName
FROM Student
WHERE sID IN (SELECT sID FROM Apply WHERE major=='CS') AND NOT sID IN (SELECT sID FROM Apply WHERE major=='EE')



# exists construct . Colleges which lie in the same state

SELECT cName,state
FROM College C1
WHERE EXISTS (SELECT * FROM College C2 WHERE C2.State=C1.State AND C1.cName<>C2.Cname)


OR MAYBE ?

SELECT cName ,state
FROM College C1, College C2
WHERE C1.state=C2.state AND C1.Cname<>C2.Cname




# college with highest enrollment  (instead of using max, we use exists)

SELECT cName
FROM College C1
WHERE NOT EXISTS (SELECT * FROM COLLEGE C2 WHERE C1.enrollment> C2.enrollment )


ORRR

SELECT cName
FROM College C1
WHERE Enrollment > ALL (SELECT Enrollmenet FROM College C2 WHERE College C1.cName <> C2.cName)

ORRR
## Any Keyword . 

Select cName
FROM College C1
WHERE NOT Enrollment <= ANY (SELECT Enrollment FROM College C2 WHERE College C1.cName<> C2.Cname)

## size of high school (all students not from the smallest high school)

SELECT sID,sName,sizeHS
FROM Student
WHERE sizeHS > ANY(SELECT sizeHS FROM Student)

using EXISTS


SELECT sID,sName,sizeHS
FROM Student S1
WHERE EXISTS (SELECT * FROM Student S22 WHERE S2.sizeHS < S1.sizeHS)


# highest GPA

SELECT sName
FROM Student S1
WHERE NOT EXISTS (SELECT * FROM Student S2 WHERE S2.GPA>S1.GPA)

ORRR

SELECT sName,GPA
FROM Student
WHERE GPA>=ALL( SELECT GPA FROM Student)







SyntaxError: invalid syntax (1808906363.py, line 4)

# Subqueries in FROM and SELECT


In [11]:
# return all students where scaling GPA changes it's value more than one

SELECT sID,sName,GPA,GPA*(sizeHS/1000) AS scaledGPA
FROM Student
WHERE GPA*(sizeHS/1000)-GPA>1 OR GPA-GPA*(sizeHS/1000)-GPA


#modifying (we'r using the formula two times in the WHERE clause)
SELECT sID,sName,GPA,GPA*(sizeHS/1000) AS scaledGPA
FROM Student
WHERE ABS(GPA*(sizeHS/1000)-GPA>1)


# modifying (still, we are using the formula again in SELECT)

SELECT*
FROM (SELECT sID,sName,GPA,GPA*(sizeHS/1000) AS scaledGpa FROM Student) G
WHERE ABS(G.scaledGPA-GPA) >1


#find colleges and pair those colleges with with the highest GPA among their applicants 


SELECT College.cName, state, GPA
FROM College,Apply,Student
WHERE College.cName=Apply.cName AND Apply.sID=Student.sID AND GPA >=ALL(SELECT GPA FROM Student,APPLY WHERE Student.sID=Apply.sID AND Apply.cName=College.cName)


Now lets write it using subquery In select clause
Subquery In select clause performs a computation, that computation should RETURN exactly one value which gets used IN result tupli

SELECT Cname,state,
(SELECT GPA FROM Apply,Student WHERE  College.cName=Apply.cName AND Apply.sID=Student.sID AND GPA >=ALL(SELECT GPA FROM Student,APPLY WHERE Student.sID=Apply.sID AND Apply.cName=College.cName) AS GPA

FROM College



SyntaxError: invalid syntax (3428525015.py, line 3)

# JOINS IN SQL


In [12]:
#1. Inner Join
# students and their majors

SELECT sName,major
FROM Student,Apply
WHERE Student.sID=Apply.sId

OR Using Inner Join

Select sName,major
FROM Student inner join Apply on Student.sId=Apply.sId


# students who applied to major CS at Stanford and less sizeHS

SELECT sName,Gpa
FROM Student,Apply
WHERE Student.sId=Apply.sId AND major='CS' AND cName='Stanford' AND sizeHS<1000

OR Using inner join 

SELECT sName, GPA
FROM Student JOIN Apply 
ON Student.sId=Apply.sId
WHERE major='CS' AND cName='Stanford' AND sizeHS<1000

# we can write conditions in ON clause instead of WHERE

SELECT sName, GPA
FROM Student JOIN Apply 
ON Student.sId=Apply.sId
AND major='CS' AND cName='Stanford' AND sizeHS<1000


# Join all three relations

SELECT Apply.sID, sName, GPA, Apply.cName,enrollment
FROM Apply,Student,College
WHERE Apply.sId=Student.sId AND Apply.cName=College.cName

OR USING JOIN

SELECT Apply.sID, sName, GPA, Apply.cName,enrollment
FROM Apply JOIN Student JOIN College
ON Apply.sId=Student.sId AND Apply.cName=College.cName

# some systems might get error on above query so modify


SELECT Apply.sID, sName, GPA, Apply.cName,enrollment
FROM (Apply JOIN Student ON Apply.sId=Student.sID) join College ON Apply.cName=College.cName




SyntaxError: invalid syntax (1254750085.py, line 4)

<li>Natural Join

In [13]:
#Natural join combines two relations which have two common column names (Student.sID, Apply.sID)

SELECT sName,major
FROM Student,Apply
WHERE Student.sID=Apply.sId   

SAME AS

SELECT sName,major
FROM Student NATURAL JOIN Apply


# instead of ON , USING is a better practice
SELECT sName,major 
FROM Student JOIN Apply USING(sID)



SyntaxError: invalid syntax (108449316.py, line 3)

In [14]:
# Joining two instances of the same relation
#pairing students with the same gpa

SELECT S1.sID, S1.name, S2.sID, S2.name
FROM Student S1 , Student S2
WHERE S1.GPA= S2.GPA AND S1.sID < S2.sID (Avoiding same student)

USING JOIN 

SELECT S1.sID, S1.name, S2.sID, S2.name
FROM Student S1 JOIN Student S2 USING (GPA)
ON S1.sID < S2.sID

# xxx above query won't work cause you cannot have USING and ON together. Thus , we use where clause

SELECT S1.sID, S1.name, S2.sID, S2.name
FROM Student S1 JOIN Student S2 USING (GPA)
WHERE S1.sID < S2.sID
                                          
                                           





SyntaxError: invalid syntax (1828088576.py, line 4)

# Outer Join

In [ ]:
# let's start with a simple join 

SELECT sName,sID,cName,major
FROM Student INNER JOIN Apply using(sID)

#let's find students who haven't applied yet 

SELECT sName,sID,cName,major
FROM Student OUTER LEFT JOIN Apply using(sID)

#the query returns sName,sID'S and cName(NULL), major(NULL) of students who applied nowehere yet. Also,can we do it without outer join?


SELECT sName,Student.sID,cName,major
FROM Student,Apply
WHERE Student.sId=Apply.sID
UNION
SELECT sName,sID,NULL,NULL
FROM Student 
WHERE sID NOT IN (SELECT sID FROM Apply)

#right outer join 

SELECT sName,sID,cName,major
FROM Student OUTER RIGHT JOIN Apply using(sID)
   
#full outer join

SELECT sName,sId,cName,major
FROM Student FULL OUTER JOIN Apply using (sId)


#writing above without full outer ie full outer= left outer + right outer

SELECT sName,sId,cName,major
FROM Student LEFT OUTER JOIN Apply using (sId)
UNION
SELECT sName,sId,cName,major
FROM Student RIGHT OUTER JOIN Apply using(sId)

#you might feel we are double counting but union operator automatically deletes duplicates
# now can we do without any joins?

SELECT sName,Student.sId,cName,major
FROM Student,Apply
WHERE Student.sId=Apply.sId
UNION 
SELECT sName,sId,NULL,NULL
FROM Student
WHERE sId NOT IN (SELECT sID FROM Apply)
UNION
Select Null,cName,major
FROM Apply
WHERE sId NOT IN (SELECT sID FROM Student)


# Aggregation

In [1]:
# average gpa

SELECT AVG(gpa)
FROM Students


#Min gpa of students who have applied to CS

SELECT MIN(gpa)
FROM Student,Apply
WHERE Student.sId=Apply.sID AND major='CS'



#Avg 

SELECT MIN(gpa)
FROM Student,Apply
WHERE Student.sId=Apply.sID AND major='CS'

#the problem with above query is that students might have applied to CS twice or more meaning avg is not accurate
# we use subquery to find accurate average

SELECT AVG(GPA)
FROM Student
WHERE sID IN (SELECT sID FROM Apply WHERE major='CS')


#count

SELECT COUNT(*)
FROM College
WHERE enrollment>1500


#students who applied to cornell

SELECT COUNT(*)
FROM Apply
WHERE Cname='Cornell'

#now similear to previous example, we are overcounting, but instead of subquery, we use DISTINCT

SELECT COUNT(DISTINCT sID)
FROM Apply
WHERE Cname='Cornell'




# the below query computes where the number of other students who have the same GPA as the student is equal to the number of 
# other student that have the same high school size 

SELECT *
FROM Student S1
WHERE (SELECT COUNT(*) FROM Student S2 WHERE S1.sId<>S2.sI AND S1.gpa=S2.gpa)
=
(SELECT COUNT(*) FROM Student S2 WHERE S1.sID<>S2.sID AND S1.sizeHS=S2.sizeHS)




#subqueries in from clause allows us to use a selectfromwhere expression and use it's result as if it were an actual table in db

#below query computes amount by which average GPA of students who applied to CS
# exceeds average gpa of students who didn't apply to CS (assuming that the average does exceed)


SELECT CS.avgGPA- NonCS.avgGPA
FROM (SELECT AVG(GPA) AS avgGPA FROM Student WHERE sID IN (SELECT sID FROM Apply WHERE major=='CS')) AS CS,
(SELECT AVG(GPA) AS avgGPA FROM Student WHERE sID NOT IN (SELECT sID FROM Apply WHERE major=='CS')) AS NonCS


#can we write above using subquery in SELECT?
# Remember, we can use subquery in select as long as it returns a single value


SELECT 
(SELECT AVG(GPA) AS avgGPA FROM Student WHERE sID IN (SELECT sID FROM Apply WHERE major=='CS')) -
(SELECT AVG(GPA) AS avgGPA FROM Student WHERE sID NOT IN (SELECT sID FROM Apply WHERE major=='CS')) AS d 
FROM Student 

# we'll get duplicates hence add distinct 

SyntaxError: invalid syntax (2732289455.py, line 3)

# Group By clause

- Only used in conjunction with aggregation

In [2]:
# number of applicants to each college

SELECT cName,count(*)
FROM Apply
GROUP BY cName

# total enrollment of college students for each state

SELECT state, SUM(enrollment)
FROM College 
GROUP BY state


#this query computes for each college and major combination
#the minimum and maximum GPAs for the students who've applied to that college.

SELECT cName,major,MIN(GPA),MAX(GPA)
FROM Student,Apply
WHERE Student.sID=Apply.sID
GROUP BY cName,major

# let's find the spread of GPA (MAX-MIN)

SELECT mx-mn
FROM (SELECT cName,major,MIN(GPA) AS mn,MAX(GPA) AS mx
FROM Student,Apply
WHERE Student.sID=Apply.sID
GROUP BY cName,major) M

# largest spread

SELECT MAX(mx-mn)
FROM (SELECT cName,major,MIN(GPA) AS mn,MAX(GPA) AS mx
FROM Student,Apply
WHERE Student.sID=Apply.sID
GROUP BY cName,major) M



#SUBTELTIES IN GROUP BY 
# below query finds out number of colleges a student has applied to 

SELECT Student.sID, COUNT(DISTINCT cName)
FROM Student,APPLY
WHERE Student.sID=Apply.sID
GROUP BY Student.sID

# what if I want to add students who did not apply to any college


SELECT Student.sID, COUNT(DISTINCT cName)
FROM Student,APPLY
WHERE Student.sID=Apply.sID
GROUP BY Student.sID
UNION
SELECT sID,0
FROM Student
WHERE sID NOT IN (SELECT sID FROM Apply)




SyntaxError: invalid syntax (3244904965.py, line 3)

# Having clause
- Also only used in conjunction with aggregation
- allows us to apply conditions on results of aggregate functions
- used after GroupBy and allows us to check conditions that involve the entire group. In contrast to WHERE which applies on one tuple at a time

In [3]:
# finding colleges with fewer than five applicaations

SELECT cName
FROM Apply
GROUP BY cName
HAVING COUNT(*)<5

# another way 

SELECT cName
FROM Apply A1
WHERE 5> (SELECT COUNT(*) FROM APPLY A2 WHERE A2.cName=A2.cName)

#will get duplicate so use distinct

# finding colleges with fewer than five applicaations

SELECT cName
FROM Apply
GROUP BY cName
HAVING COUNT(DISTINCT sID)<5


# find all majors where the maximum GPA of a student applying for that major is lower than the average GPA in the database

SELECT major
FROM Student,Apply
WHERE Student.sID=Apply.sID
GROUP BY major
HAVING MAX(GPA) < (SELECT AVG(GPA) FROM Student)


SyntaxError: invalid syntax (1987143445.py, line 3)

# Null values

In [4]:
# let's say we introduce Kevin and Lori with no GPA (null) in our data

SELECT sID,sName,GPA
FROM Student
WHERE GPA>3.5

#won't get Kevin and Lori


SELECT sID,sName,GPA
FROM Student
WHERE GPA>3.5 OR GPA<=3.5

# Still we don't get Kevin and Lori , to get them we use

SELECT sID,sName,GPA
FROM Student 
WHERE GPA>3.5 OR GPA<=3.5 OR GPA IS NULL


SyntaxError: invalid syntax (2348423744.py, line 3)

# Data modification statements INSERT

 Insert can be done in two ways




In [2]:
# 1. 
#INSERT INTO table_name VALUES (value1, value2, value3 ....)

#2. Insert using subquery
#INSERT INTO table_name SELECT statement

#3. 
#INSERT INTO table_name SELECT statement
# ie the select table will produce a set of tuples , and as long as it has same schema as the table, we can insert
# all of the tuples into the table


# ADD cmu to the colleges

INSERT INTO College values('CMU','PA',11500)


# let's make sure those students who haven't applied elsewhere yet apply to CMU


INSERT INTO Apply
SELECT sID,'Carnegie Mellon','CS',null
FROM Student
WHERE sID NOT IN (SELECT sID FROM Apply)



#first found out all students who didnt apply anywhere,then put cmu , cs , 'null' because we don't know decision yet





#Let's find students who have applied for an EE Major at other colleges and have been turned down and let's 
#have them apply to Carnegie Mellon and let's accept them right away.



INSERT INTO Apply
SELECT sID , 'Carnegie Mellon','EE','Y'
FROM Student 
WHERE sID IN (SELECT sID FROM Apply WHERE major='EE' AND decision='N')


SyntaxError: invalid syntax (456241051.py, line 15)

# Delete command

In [ ]:
# DELETE FROM table_name WHERE condition


#What we're going to do in this example is we're going to find all students that have applied to more than two different
#majors, and we're going to decide that they are unreliable students and we are going to entirely delete them from the database.


DELETE FROM Student WHERE sID IN
(SELECT sID 
FROM Apply
GROUP BY sID
HAVING Count(distinct major)>2)

#sometimes some db systems don't allow deletion where subquery and outer query has same relation



#In this example, we have
#decided that any college that doesn't
#have a CS applicant is
#probably not worth keeping in the database and we'll delete it.


DELETE * FROM College
WHERE cName NOT IN (SELECT cName FROM Apply WHERE major='CS')


# We'r going to find students whose GPA is less than three point
#six and who have applied to Carnegie Mellon.
#We're going to accept those students, #but we're going to turn them into economics majors.

UPDATE Apply
SET DECISION='Y',major='economics'
WHERE cName='Carnegie Mellon'
WHERE sID IN (SELECT sID FROM Student WHERE GPA<3.6)


#In this command we're going to find the student who has the highest GPA and has
#applied to major in EE and we're going to change
#them from an EE major to a CSE major, rather than
#just having them waste their time on electrical engineering.


UPDATE Apply
SET major='EE'
WHERE major='EE'
AND sID IN 
(SELECT sID FROM Student WHERE GPA>=ALL(SELECT GPA FROM Student WHERE sID IN (SELECT sID FROM Apply WHERE major='EE')))


#give every student the highest
#GPA and the smallest high school in the database.

UPDATE Student
SET GPA=(SELECT MAX(GPA) FROM Student),
sizeHS=(SELECT MIN(GPA) FROM Student)

#in the set command, the rhs of equals can itself be a sub query

#let's accept everybody pal

UPDATE APPLY
SET decision='Y'




